In [701]:
import requests
import copy
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname=r'./module/wqy-microhei.ttc') #設定中文字體
import re #python 的模塊 regular expression

In [702]:
'''讓使用者輸入想看哪一隊伍的球員'''
userinput=2 # 0=Lamigo, 1=統一7-ELEVEn, 2=富邦, 3=中信兄弟
teamName=['Lamigo', '統一7-ELEVEn', '富邦', '中信兄弟']
teamCode=['A02','L01','B04','E02']
res = requests.get("http://www.cpbl.com.tw/web/team_player.php?&team={}".format(teamCode[userinput]))# 抓資料頁面
soup = BeautifulSoup(res.text)
player = soup.select('td a')

In [704]:
'''查看某一隊伍所有球員的球員編號'''
playerId={}
for index,value in enumerate(player):
    aa=re.search(r'player_id=(.*)&', str(value), re.M|re.I)
    playerId[player[index].text] = aa[1]
playerId

{'伍鐸': 'E0L7',
 '羅嘉仁': 'R390',
 '陳鴻文': 'R669',
 '黃亦志': 'B0J4',
 '倪福德': 'R081',
 '蔡明晋': 'R107',
 '喬登': '004631',
 '羅力': 'A0C0',
 '賴鴻誠': 'R354',
 '郭泓志': 'R675',
 '陳仕朋': 'B0K0',
 '林羿豪': 'R670',
 '方克偉': 'R253',
 '林宥穎': 'R398',
 '蔡友達': 'B0I5',
 '林益全': 'R205',
 '范國宸': 'k509',
 '王正棠': 'k462',
 '陳品捷': 'H400',
 '李宗賢': 'H593',
 '蔣智賢': 'R399',
 '于孟馗': 'H329',
 '陳凱倫': 'R498',
 '張正偉': 'R457',
 '胡金龍': 'R678',
 '高國輝': 'R400',
 '林哲瑄': 'R401',
 '高孝儀': 'B0I0'}

In [575]:
'''請使用者輸入要找哪位打者的資料'''
inputPlayer="胡金龍"

In [576]:
'''找出這位打者2018所有的比賽編號跟日期'''
res = requests.get("http://www.cpbl.com.tw/players/follow.html?player_id={}&teamno={}&year=2018".format(playerId[inputPlayer],teamCode[userinput]))# 抓資料頁面
soup = BeautifulSoup(res.text)
gamedate=[]#存放2018比賽日期
gameId=[]#存放2018比賽編號
for trValue in soup.select('.gap_b20 table tr'):
    game = trValue.select('td')
    if game!=[]:
        dd=re.search(r"(\d\d)/(\d\d)\((\d*)\)", str(game[0].text))
        gamedate.append("2018-"+dd.group(1)+"-"+dd.group(2))
        gameId.append(dd.group(3))
print("gamedate:",gamedate)
print("gameId",gameId)


In [690]:
'''抓每一場比賽(gameId)的所有紀錄,並抓取關鍵字來看打者面對不同投手的打擊狀況'''
playRecord={} # 儲存打者的表現及面對的投手
pitcherTeam={}#儲存投手的隊伍
for indexDate,date in enumerate(gamedate):
    res = requests.get("http://www.cpbl.com.tw/games/play_by_play.html?&game_type=01&game_id={}&game_date={}&pbyear=2018".format(gameId[indexDate],date ))# 抓資料頁面
    soup = BeautifulSoup(res.text)
    player = soup.select('.gap_b20 td')
    pitcher="ERROR: Can't find pitcher"
    pTeam="ERROR: Can't find team"
    for valuePlayer in player:
        if (bool(re.search('先發投手',valuePlayer.text)) or bool(re.search('更換投手',valuePlayer.text))) and (not bool(re.search('{}'.format(teamName[userinput]),valuePlayer.text))) :
        # 抓打者所面對的投手是誰
            pitcher = re.search('投手.([^， 。,、]*)',valuePlayer.text).group(1)
            if len(pitcher)==1:
                p = re.search('投手.([^， 。,、]*)[， 。,、]([^， 。,、]*)',valuePlayer.text)
                pitcher=p.group(1)+p.group(2)
            pTeam= re.match('(.*)..投手',valuePlayer.text).group(1)
        elif  bool(re.search('{}'.format(inputPlayer),valuePlayer.text)) and (not bool(re.search('{}'.format(teamName[userinput]),valuePlayer.text))):
        # 抓指定打者(ex.胡金龍)在打擊時的紀錄
            if re.search("四壞|觸身|全壘打|三振|安打|飛球|高飛接殺|滾地球|反彈球|刺殺|軟弱滾地|野手選擇",valuePlayer.text):
            # row中出現這些字其中之一
                matchWord=re.search("四壞|觸身|全壘打|三振|安打|飛球|高飛接殺|滾地球|反彈球|刺殺|軟弱滾地|野手選擇",valuePlayer.text).group()
                if playRecord.get(pitcher)==None:
                    playRecord[pitcher]={"四壞":0,"觸身":0,"全壘打":0,"三振":0,"安打":0,"飛球":0,"滾地球":0}
                    pitcherTeam[pitcher]=pTeam
                if matchWord=="四壞" or matchWord=="觸身" or matchWord=="全壘打" or matchWord=="三振" or matchWord=="安打" :
                    playRecord[pitcher][matchWord]+=1
                elif matchWord=="飛球" or matchWord=="高飛接殺":
                    playRecord[pitcher]["飛球"]+=1
                elif matchWord=="滾地球" or matchWord=="反彈球" or matchWord=="刺殺" or matchWord=="軟弱滾地" or matchWord=="野手選擇":
                    playRecord[pitcher]["滾地球"]+=1
                else:
                    print("Error:有搜尋到那些字matchWord有誤",valuePlayer,matchWord,"gameDate:",date)
            else:
                print(valuePlayer,"gameDate:",date)


<td align="left">第三棒DH胡金龍：犧牲觸擊，二三壘有人，一出局。</td> gameDate: 2018-09-29
<td align="left">第三棒LF胡金龍：觸擊成功，二三壘有人，一出局。</td> gameDate: 2018-09-21
<td align="left">第五棒，LF，胡金龍：中外野方向高飛犧牲打，一出局，1分打點，二壘跑者推進至三壘</td> gameDate: 2018-06-12
<td align="left">第五棒DH胡金龍：犧牲觸擊，一出局，二壘有人。</td> gameDate: 2018-06-09
<td align="left">胡金龍盜壘成功，二壘有人。</td> gameDate: 2018-06-01
<td align="left">第五棒DH胡金龍：高飛犧牲打，帶有一分打點，蔣智賢前進二壘，二壘有人。</td> gameDate: 2018-06-01
<td align="left">胡金龍盜壘失敗，一出局。</td> gameDate: 2018-05-27
<td align="left">胡金龍盜二壘失敗，二出局</td> gameDate: 2018-05-12
<td align="left">第三棒LF胡金龍：一壘跑者盜壘失敗，三出局。</td> gameDate: 2018-05-04
<td align="left">第二棒 LF 胡金龍：成功的犧牲觸擊，一出局，跑者推進上到二壘</td> gameDate: 2018-04-29


In [691]:
# 胡金龍2018打擊紀錄(犧牲盜壘未紀錄)
playRecord

{'紐維拉': {'四壞': 0, '觸身': 1, '全壘打': 1, '三振': 1, '安打': 3, '飛球': 4, '滾地球': 1},
 '艾迪頓': {'四壞': 5, '觸身': 0, '全壘打': 1, '三振': 4, '安打': 3, '飛球': 7, '滾地球': 4},
 '萊福力': {'四壞': 0, '觸身': 0, '全壘打': 2, '三振': 2, '安打': 5, '飛球': 4, '滾地球': 8},
 '林樺慶': {'四壞': 0, '觸身': 0, '全壘打': 1, '三振': 1, '安打': 7, '飛球': 2, '滾地球': 2},
 '陳禹勳': {'四壞': 1, '觸身': 0, '全壘打': 0, '三振': 0, '安打': 8, '飛球': 1, '滾地球': 0},
 '瑞安': {'四壞': 1, '觸身': 0, '全壘打': 0, '三振': 3, '安打': 5, '飛球': 5, '滾地球': 6},
 '王鏡銘': {'四壞': 0, '觸身': 0, '全壘打': 0, '三振': 0, '安打': 2, '飛球': 2, '滾地球': 2},
 '江辰晏': {'四壞': 0, '觸身': 0, '全壘打': 0, '三振': 3, '安打': 2, '飛球': 8, '滾地球': 2},
 '黃竣彥': {'四壞': 0, '觸身': 0, '全壘打': 0, '三振': 0, '安打': 2, '飛球': 2, '滾地球': 1},
 '安狄': {'四壞': 1, '觸身': 0, '全壘打': 0, '三振': 0, '安打': 1, '飛球': 0, '滾地球': 0},
 '傅于剛': {'四壞': 0, '觸身': 0, '全壘打': 0, '三振': 0, '安打': 1, '飛球': 2, '滾地球': 1},
 '謝榮豪': {'四壞': 0, '觸身': 0, '全壘打': 0, '三振': 1, '安打': 0, '飛球': 1, '滾地球': 0},
 '陳琥': {'四壞': 1, '觸身': 0, '全壘打': 1, '三振': 1, '安打': 3, '飛球': 1, '滾地球': 6},
 '鄭凱文': {'四壞': 0, '觸身': 0, '

In [694]:
# 胡金龍2018所面對的投手所屬隊伍
pitcherTeam

{'紐維拉': '中信兄弟',
 '艾迪頓': '中信兄弟',
 '萊福力': '中信兄弟',
 '林樺慶': 'Lamigo桃猿',
 '陳禹勳': 'Lamigo桃猿',
 '瑞安': '統一7-ELEVEn獅',
 '王鏡銘': '統一7-ELEVEn獅',
 '江辰晏': '統一7-ELEVEn獅',
 '黃竣彥': '統一7-ELEVEn獅',
 '安狄': '統一7-ELEVEn獅',
 '傅于剛': '統一7-ELEVEn獅',
 '謝榮豪': '中信兄弟',
 '陳琥': '中信兄弟',
 '鄭凱文': '中信兄弟',
 '王溢正': 'Lamigo桃猿',
 '吳丞哲': 'Lamigo桃猿',
 '黃子鵬': 'Lamigo桃猿',
 '史博威': 'Lamigo桃猿',
 '施子謙': '統一7-ELEVEn獅',
 '林子崴': '統一7-ELEVEn獅',
 '陳韻文': '統一7-ELEVEn獅',
 '羅里奇': '統一7-ELEVEn獅',
 '邱浩鈞': '統一7-ELEVEn獅',
 '周磊': '中信兄弟',
 '陳柏豪': '中信兄弟',
 '彭識穎': '中信兄弟',
 '官大元': '中信兄弟',
 '鄭錡鴻': '中信兄弟',
 '蘇俊羽': 'Lamigo桃猿',
 '尼克斯': 'Lamigo桃猿',
 '李振昌': '中信兄弟',
 '鄭承浩': 'Lamigo桃猿',
 '林威志': '統一7-ELEVEn獅',
 '江承峰': '統一7-ELEVEn獅',
 '王躍霖': 'Lamigo桃猿',
 '69黃子鵬': 'Lamigo桃猿',
 '吳哲源': '中信兄弟',
 '洪心騏': '統一7-ELEVEn獅',
 '王玉譜': '統一7-ELEVEn獅',
 '賴泊凱': '統一7-ELEVEn獅',
 '馬丁尼茲': '統一7-ELEVen獅',
 '克恩三世': 'Lamigo Monkeys',
 '林國裕': 'Lamigo Monkeys',
 '邱品睿': '中信兄弟',
 '楊志龍': '中信兄弟',
 '黃偉晟': 'Lamigo Monkeys',
 '賽格威': '中信兄弟',
 '朱俊祥': 'Lamigo桃猿',
 '洪宸宇': '中信兄弟',
 '陽建福': '統一7-ELEVen

In [697]:
#匯出成CSV檔
import csv
with open('hu15.csv', 'w', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['投手','投手隊伍','四壞', '觸身', '全壘打','三振', '安打', '飛球','滾地球']
    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # 寫入第一列的欄位名稱
    writer.writeheader()
    # 寫入資料
    for p in playRecord.keys():
        pValue=copy.deepcopy(playRecord[p])
        pValue['投手']=p
        pValue['投手隊伍']=pitcherTeam[p]
        writer.writerow(pValue)